<a href="https://colab.research.google.com/github/blackbudge98-cpu/gt-markets/blob/main/DataProcessingSteps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2. Processing the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime, timedelta

today = datetime.now()
ninety_days_ago = today - timedelta(days=90)

filename = f"{today.strftime('%Y-%m-%d')}_{ninety_days_ago.strftime('%Y-%m-%d')}_Merged_raw_data_base.csv"

print(filename)

2025-08-30_2025-06-01_Merged_raw_data_base.csv


In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from datetime import date, timedelta
import io
import os

# Authenticate and mount Google Drive (assuming it's not already mounted and authenticated)
# If you have already run the cell S_kM5CWcaq_D, you can comment out these lines
# drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Define the folder ID
folder_id = '1tqNeIkQM2IawFLS-NHBzaDl8fsJAo0t_' # Replace with your actual folder ID

# Define the filename based on the date range
today = date.today()
ninety_days_ago = today - timedelta(days=90)
filename = f"{today.strftime('%Y-%m-%d')}_{ninety_days_ago.strftime('%Y-%m-%d')}_Merged_raw_data_base.csv"
filepath = f"/content/{filename}" # Save locally first

try:
    # Search for the file within the specific folder by name
    query = f"name='{filename}' and '{folder_id}' in parents"
    results = drive_service.files().list(q=query,
                                         spaces='drive',
                                         fields='files(id, name)').execute()
    items = results.get('files', [])

    if not items:
        print(f'No file found with name: {filename} in folder ID: {folder_id}.')
    else:
        file_id = items[0]['id']
        print(f'Found file: {items[0]["name"]} with ID: {file_id}')

        # Download the file
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")

        # Save the downloaded file to the local environment
        with open(filepath, 'wb') as f:
            fh.seek(0)
            f.write(fh.read())

        print(f"File downloaded successfully to: {filepath}")

        # You can now read the file using pandas
        import pandas as pd
        df = pd.read_csv(filepath)
        print("File loaded into pandas DataFrame:")
        display(df.head())

except Exception as e:
    print(f"An error occurred: {e}")

Found file: 2025-08-30_2025-06-01_Merged_raw_data_base.csv with ID: 1xv-RWPhVqGSMolO5fhEfdbbb350c-bIT
Download 100%.
File downloaded successfully to: /content/2025-08-30_2025-06-01_Merged_raw_data_base.csv
File loaded into pandas DataFrame:


,Date,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,...,intervention,gamble,luxury,isPartial_y,blackmail,privilege,ghetto,extravagant,nobility,isPartial
0,2025-06-01,105652.101562,NaN,NaN,NaN,NaN,4,42,0,3,...,0,0,13,False,0,0,0,0,0,False
1,2025-06-02,105881.531250,62.520000,3370.600098,56.934940,7.2037,6,78,0,5,...,0,0,15,False,0,0,0,0,0,False
2,2025-06-03,105432.468750,63.410000,3350.199951,60.107414,7.2037,5,76,0,5,...,0,0,14,False,0,0,0,0,0,False
3,2025-06-04,104731.984375,62.849998,3373.500000,60.985332,7.2037,5,71,0,5,...,0,0,16,False,0,0,0,0,0,False
4,2025-06-05,101575.953125,63.369999,3350.699951,60.057533,7.2037,6,76,0,4,...,0,0,21,False,0,0,0,0,0,False


In [ ]:
# Create a new dataframe by dropping the 'isPartial' column
edited_dataframe = df.drop(columns=['isPartial','isPartial_y','isPartial_x'])

# Print the head of the new dataframe
print("Head of edited_dataframe:")
display(edited_dataframe.head())

Head of edited_dataframe:


,Date,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,Donald Trump,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility
0,2025-06-01,105652.101562,NaN,NaN,NaN,NaN,4,42,0,3,1,29,0,0,13,0,0,0,0,0
1,2025-06-02,105881.531250,62.520000,3370.600098,56.934940,7.2037,6,78,0,5,1,40,0,0,15,0,0,0,0,0
2,2025-06-03,105432.468750,63.410000,3350.199951,60.107414,7.2037,5,76,0,5,1,40,0,0,14,0,0,0,0,0
3,2025-06-04,104731.984375,62.849998,3373.500000,60.985332,7.2037,5,71,0,5,1,39,0,0,16,0,0,0,0,0
4,2025-06-05,101575.953125,63.369999,3350.699951,60.057533,7.2037,6,76,0,4,2,51,0,0,21,0,0,0,0,0


In [ ]:
# Convert the 'Date' column to datetime objects
edited_dataframe['Date'] = pd.to_datetime(edited_dataframe['Date'])

# Add a new column for the day of the week
edited_dataframe['Day of Week'] = edited_dataframe['Date'].dt.day_name()

# Display the head of the updated dataframe
print("Head of edited_dataframe with 'Day of Week' column:")
display(edited_dataframe.head())

Head of edited_dataframe with 'Day of Week' column:


,Date,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,...,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility,Day of Week
0,2025-06-01,105652.101562,NaN,NaN,NaN,NaN,4,42,0,3,...,29,0,0,13,0,0,0,0,0,Sunday
1,2025-06-02,105881.531250,62.520000,3370.600098,56.934940,7.2037,6,78,0,5,...,40,0,0,15,0,0,0,0,0,Monday
2,2025-06-03,105432.468750,63.410000,3350.199951,60.107414,7.2037,5,76,0,5,...,40,0,0,14,0,0,0,0,0,Tuesday
3,2025-06-04,104731.984375,62.849998,3373.500000,60.985332,7.2037,5,71,0,5,...,39,0,0,16,0,0,0,0,0,Wednesday
4,2025-06-05,101575.953125,63.369999,3350.699951,60.057533,7.2037,6,76,0,4,...,51,0,0,21,0,0,0,0,0,Thursday


In [ ]:
# Define the list of weekend days
weekend_days = ['Saturday', 'Sunday']

# Drop rows where 'Day of Week' is in the weekend_days list
edited_dataframe_weekdays = edited_dataframe[~edited_dataframe['Day of Week'].isin(weekend_days)].copy()

# Display the head of the new dataframe with only weekdays
print("Head of edited_dataframe with only weekdays:")
display(edited_dataframe_weekdays.head())

Head of edited_dataframe with only weekdays:


,Date,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,...,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility,Day of Week
1,2025-06-02,105881.531250,62.520000,3370.600098,56.934940,7.2037,6,78,0,5,...,40,0,0,15,0,0,0,0,0,Monday
2,2025-06-03,105432.468750,63.410000,3350.199951,60.107414,7.2037,5,76,0,5,...,40,0,0,14,0,0,0,0,0,Tuesday
3,2025-06-04,104731.984375,62.849998,3373.500000,60.985332,7.2037,5,71,0,5,...,39,0,0,16,0,0,0,0,0,Wednesday
4,2025-06-05,101575.953125,63.369999,3350.699951,60.057533,7.2037,6,76,0,4,...,51,0,0,21,0,0,0,0,0,Thursday
5,2025-06-06,104390.343750,64.580002,3322.699951,60.057533,7.1750,5,75,0,4,...,41,0,0,20,0,0,0,0,0,Friday


In [ ]:
# Define the columns to round
columns_to_round = ['USD to BTC', 'USD to Oil', 'USD to Gold', 'USD']

# Round the specified columns to 4 decimal places
edited_dataframe_weekdays[columns_to_round] = edited_dataframe_weekdays[columns_to_round].round(4)

# Display the head of the updated dataframe
print("Head of edited_dataframe_weekdays after rounding:")
display(edited_dataframe_weekdays.head())

Head of edited_dataframe_weekdays after rounding:


,Date,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,Oil Price,...,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility,Day of Week
1,2025-06-02,105881.5312,62.52,3370.6001,56.9349,7.2037,6,78,0,5,...,40,0,0,15,0,0,0,0,0,Monday
2,2025-06-03,105432.4688,63.41,3350.2000,60.1074,7.2037,5,76,0,5,...,40,0,0,14,0,0,0,0,0,Tuesday
3,2025-06-04,104731.9844,62.85,3373.5000,60.9853,7.2037,5,71,0,5,...,39,0,0,16,0,0,0,0,0,Wednesday
4,2025-06-05,101575.9531,63.37,3350.7000,60.0575,7.2037,6,76,0,4,...,51,0,0,21,0,0,0,0,0,Thursday
5,2025-06-06,104390.3438,64.58,3322.7000,60.0575,7.1750,5,75,0,4,...,41,0,0,20,0,0,0,0,0,Friday


In [ ]:
# Get the current list of columns
current_columns = edited_dataframe_weekdays.columns.tolist()

# Define the desired order for the first two columns
new_order_start = ['Date', 'Day of Week']

# Get the remaining columns, excluding 'Date' and 'Day of Week'
remaining_columns = [col for col in current_columns if col not in new_order_start]

# Create the new column order
new_column_order = new_order_start + remaining_columns

# Reindex the DataFrame with the new column order
edited_dataframe_weekdays = edited_dataframe_weekdays[new_column_order]

# Display the head of the DataFrame with the reordered columns
print("Head of edited_dataframe_weekdays with reordered columns:")
display(edited_dataframe_weekdays.head())

Head of edited_dataframe_weekdays with reordered columns:


,Date,Day of Week,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,...,Donald Trump,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility
1,2025-06-02,Monday,105881.5312,62.52,3370.6001,56.9349,7.2037,6,78,0,...,1,40,0,0,15,0,0,0,0,0
2,2025-06-03,Tuesday,105432.4688,63.41,3350.2000,60.1074,7.2037,5,76,0,...,1,40,0,0,14,0,0,0,0,0
3,2025-06-04,Wednesday,104731.9844,62.85,3373.5000,60.9853,7.2037,5,71,0,...,1,39,0,0,16,0,0,0,0,0
4,2025-06-05,Thursday,101575.9531,63.37,3350.7000,60.0575,7.2037,6,76,0,...,2,51,0,0,21,0,0,0,0,0
5,2025-06-06,Friday,104390.3438,64.58,3322.7000,60.0575,7.1750,5,75,0,...,2,41,0,0,20,0,0,0,0,0


In [ ]:
# Display the info of the dataframe to check for non-null values and data types
print("Info for edited_dataframe_weekdays:")
edited_dataframe_weekdays.info()

Info for edited_dataframe_weekdays:
<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 1 to 89
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 65 non-null     datetime64[ns]
 1   Day of Week          65 non-null     object        
 2   USD to BTC           65 non-null     float64       
 3   USD to Oil           64 non-null     float64       
 4   USD to Gold          64 non-null     float64       
 5   USD                  63 non-null     float64       
 6   USD to Chinese Yuan  65 non-null     float64       
 7   USD price            65 non-null     int64         
 8   Gold price           65 non-null     int64         
 9   Chinese Yuan Price   65 non-null     int64         
 10  Oil Price            65 non-null     int64         
 11  Donald Trump         65 non-null     int64         
 12  inflation            65 non-null     int64         
 13  interv

In [ ]:
# Identify columns with missing values
columns_with_nulls = ['USD to Oil', 'USD to Gold', 'USD']

# Filter the DataFrame to show rows where any of the specified columns have a null value
rows_with_nulls = edited_dataframe_weekdays[edited_dataframe_weekdays[columns_with_nulls].isnull().any(axis=1)]

# Display the rows with nulls
print("Rows in edited_dataframe_weekdays with null values:")
display(rows_with_nulls)

Rows in edited_dataframe_weekdays with null values:


,Date,Day of Week,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,...,Donald Trump,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility
18,2025-06-19,Thursday,104684.2891,NaN,NaN,NaN,7.1888,4,66,0,...,1,37,0,0,19,67,0,0,0,0
33,2025-07-04,Friday,108034.3359,66.5,3332.5,NaN,7.1649,5,51,0,...,6,48,0,0,27,0,0,0,0,0


In [ ]:
# Define the columns where NaN values should be filled
columns_to_fill = ['USD to Oil', 'USD to Gold', 'USD']

# Fill NaN values with the previous day's value (forward fill) for the specified columns using .ffill()
edited_dataframe_weekdays[columns_to_fill] = edited_dataframe_weekdays[columns_to_fill].ffill()

# Display the head of the dataframe after filling NaN values
print("Head of edited_dataframe_weekdays after filling NaN values:")
display(edited_dataframe_weekdays.head())

Head of edited_dataframe_weekdays after filling NaN values:


,Date,Day of Week,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,...,Donald Trump,inflation,intervention,gamble,luxury,blackmail,privilege,ghetto,extravagant,nobility
1,2025-06-02,Monday,105881.5312,62.52,3370.6001,56.9349,7.2037,6,78,0,...,1,40,0,0,15,0,0,0,0,0
2,2025-06-03,Tuesday,105432.4688,63.41,3350.2000,60.1074,7.2037,5,76,0,...,1,40,0,0,14,0,0,0,0,0
3,2025-06-04,Wednesday,104731.9844,62.85,3373.5000,60.9853,7.2037,5,71,0,...,1,39,0,0,16,0,0,0,0,0
4,2025-06-05,Thursday,101575.9531,63.37,3350.7000,60.0575,7.2037,6,76,0,...,2,51,0,0,21,0,0,0,0,0
5,2025-06-06,Friday,104390.3438,64.58,3322.7000,60.0575,7.1750,5,75,0,...,2,41,0,0,20,0,0,0,0,0


In [ ]:
# Display the info of the dataframe to check for non-null values after filling
print("-" * 20)
print("Info for edited_dataframe_weekdays after filling NaN values:")
edited_dataframe_weekdays.info()
print("-" * 20)

# Check for any remaining null values
if edited_dataframe_weekdays.isnull().sum().sum() == 0:
    print("No null values found. It is safe to proceed.")
else:
    print("There are still null values in the DataFrame.")
    print(edited_dataframe_weekdays.isnull().sum())

--------------------
Info for edited_dataframe_weekdays after filling NaN values:
<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 1 to 89
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 65 non-null     datetime64[ns]
 1   Day of Week          65 non-null     object        
 2   USD to BTC           65 non-null     float64       
 3   USD to Oil           65 non-null     float64       
 4   USD to Gold          65 non-null     float64       
 5   USD                  65 non-null     float64       
 6   USD to Chinese Yuan  65 non-null     float64       
 7   USD price            65 non-null     int64         
 8   Gold price           65 non-null     int64         
 9   Chinese Yuan Price   65 non-null     int64         
 10  Oil Price            65 non-null     int64         
 11  Donald Trump         65 non-null     int64         
 12  inflation        

In [ ]:
# Select only numerical columns for calculating percentage change
numerical_columns = edited_dataframe_weekdays.select_dtypes(include=['float64', 'int64']).columns

# Exclude the 'Date' column from the numerical columns if it was included
if 'Date' in numerical_columns:
    numerical_columns = numerical_columns.drop('Date')

# Calculate the daily percentage change for each numerical column and add as new columns
for col in numerical_columns:
    new_col_name = f"{col}_Daily_Change%"
    edited_dataframe_weekdays[new_col_name] = edited_dataframe_weekdays[col].pct_change() * 100

# Display the head of the DataFrame with the new daily change columns
print("Head of edited_dataframe_weekdays with daily percentage change columns:")
display(edited_dataframe_weekdays.head())

Head of edited_dataframe_weekdays with daily percentage change columns:


,Date,Day of Week,USD to BTC,USD to Oil,USD to Gold,USD,USD to Chinese Yuan,USD price,Gold price,Chinese Yuan Price,...,Donald Trump_Daily_Change%,inflation_Daily_Change%,intervention_Daily_Change%,gamble_Daily_Change%,luxury_Daily_Change%,blackmail_Daily_Change%,privilege_Daily_Change%,ghetto_Daily_Change%,extravagant_Daily_Change%,nobility_Daily_Change%
1,2025-06-02,Monday,105881.5312,62.52,3370.6001,56.9349,7.2037,6,78,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-03,Tuesday,105432.4688,63.41,3350.2000,60.1074,7.2037,5,76,0,...,0.0,0.000000,NaN,NaN,-6.666667,NaN,NaN,NaN,NaN,NaN
3,2025-06-04,Wednesday,104731.9844,62.85,3373.5000,60.9853,7.2037,5,71,0,...,0.0,-2.500000,NaN,NaN,14.285714,NaN,NaN,NaN,NaN,NaN
4,2025-06-05,Thursday,101575.9531,63.37,3350.7000,60.0575,7.2037,6,76,0,...,100.0,30.769231,NaN,NaN,31.250000,NaN,NaN,NaN,NaN,NaN
5,2025-06-06,Friday,104390.3438,64.58,3322.7000,60.0575,7.1750,5,75,0,...,0.0,-19.607843,NaN,NaN,-4.761905,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get the list of all columns
all_columns = edited_dataframe_weekdays.columns.tolist()

# Separate the original numerical columns and their corresponding percentage change columns
original_numerical_columns = edited_dataframe_weekdays.select_dtypes(include=['float64', 'int64']).columns.tolist()
percentage_change_columns = [col for col in all_columns if col.endswith('_Daily_Change%')]

# Remove percentage change columns from the original numerical columns list
for p_col in percentage_change_columns:
    original_col = p_col.replace('_Daily_Change%', '')
    if original_col in original_numerical_columns:
        original_numerical_columns.remove(original_col)

# Define the initial columns (Date and Day of Week)
initial_columns = ['Date', 'Day of Week']

# Create the new ordered list of columns
new_ordered_columns = initial_columns
for col in original_numerical_columns:
    new_ordered_columns.append(col)
    percentage_change_col = f"{col}_Daily_Change%"
    if percentage_change_col in percentage_change_columns:
        new_ordered_columns.append(percentage_change_col)
        percentage_change_columns.remove(percentage_change_col) # Remove once added

# Add any remaining percentage change columns (shouldn't be any if logic is correct)
new_ordered_columns.extend(percentage_change_columns)

# Reindex the DataFrame with the new column order
edited_dataframe_weekdays = edited_dataframe_weekdays[new_ordered_columns]

# Display the head of the DataFrame with the reordered columns
print("Head of edited_dataframe_weekdays with reordered columns:")
display(edited_dataframe_weekdays.head())

Head of edited_dataframe_weekdays with reordered columns:


,Date,Day of Week,USD to BTC_Daily_Change%,USD to Oil_Daily_Change%,USD to Gold_Daily_Change%,USD_Daily_Change%,USD to Chinese Yuan_Daily_Change%,USD price_Daily_Change%,Gold price_Daily_Change%,Chinese Yuan Price_Daily_Change%,...,Donald Trump_Daily_Change%,inflation_Daily_Change%,intervention_Daily_Change%,gamble_Daily_Change%,luxury_Daily_Change%,blackmail_Daily_Change%,privilege_Daily_Change%,ghetto_Daily_Change%,extravagant_Daily_Change%,nobility_Daily_Change%
1,2025-06-02,Monday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-03,Tuesday,-0.424118,1.423544,-0.605236,5.572153,0.000000,-16.666667,-2.564103,NaN,...,0.0,0.000000,NaN,NaN,-6.666667,NaN,NaN,NaN,NaN,NaN
3,2025-06-04,Wednesday,-0.664392,-0.883141,0.695481,1.460552,0.000000,0.000000,-6.578947,NaN,...,0.0,-2.500000,NaN,NaN,14.285714,NaN,NaN,NaN,NaN,NaN
4,2025-06-05,Thursday,-3.013436,0.827367,-0.675856,-1.521350,0.000000,20.000000,7.042254,NaN,...,100.0,30.769231,NaN,NaN,31.250000,NaN,NaN,NaN,NaN,NaN
5,2025-06-06,Friday,2.770725,1.909421,-0.835646,0.000000,-0.398405,-16.666667,-1.315789,NaN,...,0.0,-19.607843,NaN,NaN,-4.761905,NaN,NaN,NaN,NaN,NaN
